In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

url="https://catalog.tamu.edu/undergraduate/course-descriptions/csce/"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html5lib')

course_title_instances = []
for s in soup.find_all("p", {"class": "courseblocktitle noindent"}):
  instance = s.get_text()
  course_title_instances.append(re.split("[0-9]", instance.replace("\n", ""))[-1].strip())

course_instances = []
for s in soup.find_all("p", {"class": "courseblocktitle noindent"}):
  instance = s.get_text().split()[:2]
  course_instances.append(instance if len(instance[1]) == 3 else [instance[0], (instance[1])[:instance[1].find("/")]])

result_filtered_instances = []
for s in soup.find_all("p", {"class": "hours noindent"}):
  repl = s.get_text().replace("Credits", "Credit")
  repl = repl.replace("Lecture Hours", "Lecture Hour")
  repl = repl.replace("Lab Hours", "Lab Hour")
  repl = repl.replace("Other Hours", "Other Hour")
  result = re.split(" \n", repl.replace("\n", ""))
  result = re.split("\. ", result[0])
  result_filtered = list(filter(None, result))
  result_filtered_instances.append(result_filtered)

course_description_instances = []
for s in soup.find_all("p", {"class": "courseblockdesc"}):
  instance = s.get_text().strip("\n").strip()
  course_description_instances.append(instance)

df = pd.DataFrame()

def getStringIndex(array, index, string):
  for i, s in enumerate(array[index]):
        if string in s:
              return i
  return None

def getDigits(array, index, string):
  instances = []
  indexes = getStringIndex(array, index, string)

  if(not indexes == None):
    instances = [re.findall("\d+", array[index][indexes])[0], 
                  re.findall("\d+", array[index][indexes])[-1]]
  else:
    instances = [None]
  return instances


df = pd.DataFrame(columns=["department", "course", "title", "credit hours (u)", "credit hours (l)", "lecture hours (u)", "lecture hours (l)", "lab hours (u)", "lab hours (l)", "other hours (u)", "other hours (l)", "description"]) 

for i in range(len(course_instances)):

  credit_hours = getDigits(result_filtered_instances, i, 'Credit')
  lecture_hours = getDigits(result_filtered_instances, i, 'Lecture')
  lab_hours = getDigits(result_filtered_instances, i, 'Lab')
  other_hours = getDigits(result_filtered_instances, i, 'Other')
  df.at[i, "department"] = course_instances[i][0]
  df.at[i, "course"] = course_instances[i][1]
  df.at[i, "title"] = course_title_instances[i]
  df.at[i, "credit hours (l)"] = credit_hours[0]
  df.at[i, "credit hours (u)"] = credit_hours[-1]
  df.at[i, "lecture hours (l)"] = lecture_hours[0]
  df.at[i, "lecture hours (u)"] = lecture_hours[-1]
  df.at[i, "lab hours (l)"] = lab_hours[0]
  df.at[i, "lab hours (u)"] = lab_hours[-1]
  df.at[i, "other hours (l)"] = other_hours[0]
  df.at[i, "other hours (u)"] = other_hours[-1]
  df.at[i, "description"] = course_description_instances[i]
df.reset_index(drop=True, inplace=True)
print(df)
df.to_csv("/content/drive/Shareddrives/CSCE 634/Software/Data/Courses/courses.csv", index=False)


   department course                                              title  \
0        CSCE    110                                      Programming I   
1        CSCE    111  Introduction to Computer Science Concepts and ...   
2        CSCE    120                        Program Design and Concepts   
3        CSCE    121        Introduction to Program Design and Concepts   
4        CSCE    181                          Introduction to Computing   
..        ...    ...                                                ...   
60       CSCE    482                             Senior Capstone Design   
61       CSCE    483                            Computer Systems Design   
62       CSCE    485                                   Directed Studies   
63       CSCE    489                               Special Topics in...   
64       CSCE    491                                           Research   

   credit hours (u) credit hours (l) lecture hours (u) lecture hours (l)  \
0                 4    